#Libs

In [ ]:
!pip install polars

In [ ]:
import polars as pl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np

In [ ]:
pl.Config.set_tbl_rows(10)
pl.Config.set_tbl_cols(None)

#Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
columns = ['ServerTimestamp [epoch]',
           'MarketTimestamp [epoch]',
           'ServerTimestamp [datatime, us]',
           'MarketTimestamp [datatime,us]',
           'Mdtype',
           '[price;qty;nborders] ask 0',
           '[price;qty;nborders] ask 1',
           '[price;qty;nborders] ask 2',
           '[price;qty;nborders] ask 3',
           '[price;qty;nborders] ask 4',
           '[price;qty;nborders] ask 5',
           '[price;qty;nborders] ask 6',
           '[price;qty;nborders] ask 7',
           '[price;qty;nborders] ask 8',
           '[price;qty;nborders] ask 9',
           'M',
           '[price;qty;nborders] bid 0',
           '[price;qty;nborders] bid 1',
           '[price;qty;nborders] bid 2',
           '[price;qty;nborders] bid 3',
           '[price;qty;nborders] bid 4',
           '[price;qty;nborders] bid 5',
           '[price;qty;nborders] bid 6',
           '[price;qty;nborders] bid 7',
           '[price;qty;nborders] bid 8',
           '[price;qty;nborders] bid 9',
           'Obflag',
           'ftflagMSBid ',
           'ftflagMSAsk ',
           'ftflagTLSell ',
           'ftflagTLBuy ',
           'Stream ',
           'Revision ']

In [ ]:
df = pl.read_csv("/content/drive/MyDrive/Quant/dataset_test/Local#FAST_CURR_MD#MOEX_CURR#CETS#USDCNY_TOM.2023-08-14.gz",
                 compression='gzip',
                 has_header=False,
                 new_columns=columns)

# Part 1

## Frequency

In [ ]:
Frequency_data = df.filter((pl.col('Mdtype') == 0) | (pl.col('Mdtype') == 2)).select(['ServerTimestamp [datatime, us]', 'Stream '])

Frequency_data = Frequency_data.with_columns(
    pl.col('ServerTimestamp [datatime, us]').str.strptime(pl.Datetime, fmt='%Y-%m-%d %H:%M:%S.%f')
)

Frequency_data = Frequency_data.sort('ServerTimestamp [datatime, us]').with_columns(
    (pl.col('ServerTimestamp [datatime, us]') - pl.col('ServerTimestamp [datatime, us]').shift(1)).dt.seconds().alias('TimeDiff')
)

Frequency_data = Frequency_data.with_columns([
    pl.col('ServerTimestamp [datatime, us]').dt.hour().alias('Hour'),
    pl.col('ServerTimestamp [datatime, us]').dt.minute().alias('Minute'),
    pl.col('ServerTimestamp [datatime, us]').dt.second().alias('Second')
])

In [ ]:
start_time = datetime.strptime('10:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('18:40:00', '%H:%M:%S').time()
filtered_data = Frequency_data.filter(
    (pl.col('ServerTimestamp [datatime, us]').dt.time() >= start_time) &
    (pl.col('ServerTimestamp [datatime, us]').dt.time() <= end_time)
)

frequency = filtered_data.groupby(['Hour', 'Minute', 'Second']).agg(pl.count().alias('count'))

frequency = frequency.with_columns(
    (pl.col('Hour').cast(pl.Utf8) + ':' +
     pl.col('Minute').cast(pl.Utf8).zfill(2) + ':' +
     pl.col('Second').cast(pl.Utf8).zfill(2)).alias('Time')
).drop(['Hour', 'Minute', 'Second']).rename({'Time': 'index'})

In [ ]:
def create_time_range(start_time, end_time, interval_minutes):
    start_dt = datetime.strptime(start_time, '%H:%M:%S')
    end_dt = datetime.strptime(end_time, '%H:%M:%S')
    interval = timedelta(minutes=interval_minutes)
    current_time = start_dt
    times = []
    while current_time <= end_dt:
        times.append(current_time)
        current_time += interval
    return times

In [ ]:
start_str = '10:00:00'
end_str = '18:40:00'
interval = 20

tick_positions = create_time_range(start_str, end_str, interval)

plt.figure(figsize=(15, 5))
plt.plot(frequency['index'], frequency['count'], color='blue', linewidth=1, linestyle='-', alpha=0.8)
plt.title('Частота обновлений данных ОВ за каждую секунду', fontsize=16, pad=20)
plt.xlabel('Время', fontsize=14, labelpad=15)
plt.ylabel('Частота обновлений', fontsize=14, labelpad=15)

# Set custom x-axis formatter to display only HH:MM:SS
plt.gca().xaxis.set_major_formatter(DateFormatter('%H:%M:%S'))
# Manually set x-axis ticks
plt.gca().set_xticks(tick_positions)

plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()

## Distance

n% переданной информации = $(1 - \frac{n}{100})$ квантиль времени обновления / 2 (в сторону получателя и обратно):

#### Оценка расстояния от сервера Московской биржи до аккаунта пользователя.

Для оценки расстояния от сервера Московской биржи до аккаунта пользователя можно использовать информацию о времени задержки передачи данных (RTT) и скорости света в оптическом волокне. Сигналы в оптическом волокне распространяются со скоростью, которая составляет примерно $\frac{2}{3}$ от скорости света в вакууме.

#### Данные для расчета:
- **Время туда и обратно (RTT):** 7 микросекунд (мкс)
- **Скорость света в вакууме:** \($c = 3 \times 10^8$\) м/с
- **Коэффициент преломления оптического волокна:** \($n \approx 1.5$\) что означает, что скорость света в оптическом волокне составляет $\frac{c}{1.5}$

#### Шаги для расчета:

1. **Определяем скорость света в оптическом волокне:**
   \begin{align}
   c_{\text{волокно}} = \frac{3 \times 10^8\ \text{м/с}}{1.5} = 2 \times 10^8\ \text{м/с}
   \end{align}

2. **Время передачи сигнала в одну сторону:**
   \begin{align}
   t_{\text{один путь}} = \frac{7 \times 10^{-6}\ \text{сек}}{2} = 3.5 \times 10^{-6}\ \text{сек}
   \end{align}

3. **Расчет расстояния:**
   \begin{align}
   d = t_{\text{один путь}} \times c_{\text{волокно}} = 700\ \text{м}
   \end{align}



In [ ]:
# Calculation of quantile values from 0.01 to 0.99
quantiles = np.linspace(0.01, 0.99, 99)
time_quantiles = Frequency_data['TimeDiff'].quantile(quantiles).to_numpy() / 2

c = 3 * 10**8  # speed of light in a vacuum (m/s)
n = 1.5  # refractive index of optical fiber
c_fiber = c / n  # speed of light in optical fiber (m/s)

# Calculate the distance for each time value (using quantile values)
distance_quantiles = time_quantiles * c_fiber

# Calculate the percentage of information loss
loss_percent = 100 * (1 - quantiles)

# Create labels for the X-axis: (time, information loss)
x_labels = [f"{t:.6f} sec\n{loss:.1f}% information" for t, loss in zip(time_quantiles, loss_percent)]

plt.figure(figsize=(40, 16))
plt.plot(range(len(time_quantiles)), distance_quantiles, marker='o', color='blue', linewidth=2, markersize=4)
plt.title('Зависимость возможного расстояния от % переданной информации', fontsize=16)
plt.xlabel('Время (секунды), Переданная информация (%)', fontsize=14)
plt.ylabel('Расстояние (метры)', fontsize=14)
plt.grid(True)
plt.yscale('log')
plt.xticks(ticks=range(len(time_quantiles)), labels=x_labels, rotation=45, ha='right', fontsize=8)

plt.tight_layout()
plt.show()

#Part 2

In [ ]:
# Filter rows where Mdtype is 1
data = df.filter(pl.col('Mdtype') == 1)

# Convert ServerTimestamp to datetime
data = data.with_column(pl.col('ServerTimestamp [datatime, us]').str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S"))

# Filter data based on time range
start_time = datetime.strptime('10:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('18:40:00', '%H:%M:%S').time()

filtered_data = data.filter(
    (pl.col('ServerTimestamp [datatime, us]').dt.time() >= start_time) &
    (pl.col('ServerTimestamp [datatime, us]').dt.time() <= end_time)
)

# Convert the column to integer and determine the side of the trade
filtered_data = filtered_data.with_columns([
    pl.col('[price;qty;nborders] ask 3').cast(pl.Int32),
    pl.when(pl.col('[price;qty;nborders] ask 3') == 1).then('buy').otherwise('sell').alias('side')
])

# Determine TradeID
filtered_data = filtered_data.with_column(
    ((pl.col('MarketTimestamp [epoch]').shift(1) != pl.col('MarketTimestamp [epoch]')) |
     (pl.col('side').shift(1) != pl.col('side')))
    .cumsum()
    .alias('TradeID')
)

# Extract the quantity
filtered_data = filtered_data.with_column(
    pl.col('[price;qty;nborders] ask 0').cast(pl.Float64).alias('Qty')
)

# Group by TradeID and sum the quantities
trade_sizes = filtered_data.groupby('TradeID').agg(
    pl.col('Qty').sum().alias('TotalQty')
)

In [ ]:
# Calculate statistics
mean_qty = trade_sizes['TotalQty'].mean()
stddev_qty = trade_sizes['TotalQty'].std()
median_qty = trade_sizes['TotalQty'].median()
percentiles = trade_sizes['TotalQty'].quantile([0.6, 0.7, 0.8, 0.9])

percentile_pairs = [(f"{int(p*100)}%", int(v)) for p, v in zip([0.6, 0.7, 0.8, 0.9], percentiles)]

print("Mean:", mean_qty)
print("StdDev:", stddev_qty)
print("Median:", median_qty)
print("\nPercentiles.")
for pair in percentile_pairs:
    print(f"{pair[0]}th Percentile: {pair[1]:.2f}")

In [ ]:
def calculate_probabilities(filtered_data, trade_sizes, threshold):
    # Filter trades by quantity threshold
    mask = trade_sizes.filter(pl.col('TotalQty') >= threshold)

    # Filter the original data by TradeID
    filtered = filtered_data.filter(pl.col('TradeID').is_in(mask['TradeID']))

    # Compare the current side with the previous one
    filtered = filtered.with_columns([
        pl.col('side').shift(1).alias('prev_side'),
        (pl.col('side') == pl.col('prev_side')).alias('side_match')
    ])

    # Count matches and calculate probability
    matching_sides = filtered['side_match'].sum()
    total_trades = filtered.height
    probability = matching_sides / total_trades if total_trades > 0 else 0
    return probability

In [ ]:
# Calculate probabilities
prob_mean = calculate_probabilities(filtered_data, trade_sizes, mean_qty)
prob_median = calculate_probabilities(filtered_data, trade_sizes, median_qty)
prob_mean_plus_stddev = calculate_probabilities(filtered_data, trade_sizes, mean_qty + stddev_qty)

print(f"Probability(Qty >= mean): {prob_mean:.4f}")
print(f"Probability(Qty >= median): {prob_median:.4f}")
print(f"Probability(Qty >= mean + stddev): {prob_mean_plus_stddev:.4f}")